#Importing Data

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2

from sklearn.preprocessing import LabelEncoder
from keras.applications.resnet import ResNet50
from keras.applications.densenet import DenseNet121
from keras.applications.efficientnet import EfficientNetB5
from keras.applications import imagenet_utils
import numpy as np
import pickle
import os
import torch
import matplotlib.pyplot as plt
from PIL import Image

import warnings
warnings.filterwarnings("ignore")

from google.colab import drive

drive.mount('/content/drive')

In [ ]:
from torchvision import datasets, transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

root_dir = '/content/drive/My Drive/data'

dataset = datasets.ImageFolder(root=root_dir, transform=transform)

#Data Augmentation

In [ ]:
import os
import cv2
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from imgaug import augmenters as iaa

root_dir = '/content/drive/My Drive/data'


class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = os.listdir(root_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.images[idx])
        image = cv2.imread(img_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = int(self.images[idx].split('_')[0])

        if self.transform:
            image = self.transform(image)

        return image, label

# Define data augmentation pipeline
augmentation = iaa.Sequential([
    iaa.SomeOf((0, 3), [
        # Your augmentation techniques here
    ])
], random_order=True)

# Define the transformation pipeline
transform = transforms.Compose([
    augmentation,
    transforms.ToTensor(),
])

# Create an instance of your custom dataset with the defined augmentation
custom_dataset = CustomDataset(root_dir=root_dir, transform=transform)

# Define the path to save the augmented dataset
save_path = '/content/drive/My Drive/custom_data'

# Create the custom_data directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

# Save the augmented dataset to a new file in your Google Drive
torch.save(custom_dataset, os.path.join(save_path, 'augmented_dataset.pth'))


In [ ]:
import os
import cv2
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from imgaug import augmenters as iaa

root_dir = '/content/drive/My Drive/data'

class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = os.listdir(root_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.images[idx])
        image = cv2.imread(img_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = int(self.images[idx].split('_')[0])

        if self.transform:
            image = self.transform(image)

        return image, label

augmentation = iaa.Sequential([
    iaa.SomeOf((0, 3), [])
], random_order=True)

transform = transforms.Compose([
    augmentation,
    transforms.ToTensor(),
])

custom_dataset = CustomDataset(root_dir=root_dir, transform=transform)

save_path = '/content/drive/My Drive/custom_data'

os.makedirs(save_path, exist_ok=True)

torch.save(custom_dataset, os.path.join(save_path, 'augmented_dataset.pth'))


#Splitting the data

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split

# root_dir2 = '/content/drive/My Drive/custom_data'
# dataset = datasets.ImageFolder(root=root_dir2, transform=transform)

In [ ]:
# Split the dataset into train, val, and test sets
dataset_size = len(dataset)
train_size = int(0.7 * dataset_size)
val_size = int(0.15 * dataset_size)
test_size = dataset_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

#Building the Model - (ResNet50)

In [ ]:
import torch
import torch.nn as nn
from torchvision.models import resnet50

class ResNet50_Transformer(nn.Module):
    def __init__(self, num_classes=5):
        super(ResNet50_Transformer, self).__init__()
        self.resnet = resnet50(pretrained=True)
        self.resnet.fc = nn.Identity()

        self.embedding = nn.Linear(2048, 512)
        self.transformer_layers = nn.TransformerEncoderLayer(d_model=512, nhead=8)
        self.transformer = nn.TransformerEncoder(self.transformer_layers, num_layers=4)
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):

        x = self.resnet(x)
        x = self.embedding(x)
        x = x.unsqueeze(1)
        x = self.transformer(x)
        x = x.mean(dim=1)
        x = self.fc(x)

        return x

#Building Vit (Vision Transformer)

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchvision.models import resnet50
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from torch.autograd import Variable
import numpy as np

class VisionTransformer(nn.Module):
    def __init__(self, input_channels=3, num_classes=5):
        super(VisionTransformer, self).__init__()
        self.embedding = nn.Linear(input_channels * 256 * 256, 512)
        self.transformer_layers = nn.TransformerEncoderLayer(d_model=512, nhead=8)
        self.transformer = nn.TransformerEncoder(self.transformer_layers, num_layers=4)
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):

        x = x.view(x.size(0), -1)
        x = self.embedding(x)
        x = x.unsqueeze(1)
        x = self.transformer(x)
        x = x.mean(dim=1)
        x = self.fc(x)

        return x

vit_model = VisionTransformer(input_channels=3, num_classes=5)
print(vit_model)

VisionTransformer(
  (embedding): Linear(in_features=196608, out_features=512, bias=True)
  (transformer_layers): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
    )
    (linear1): Linear(in_features=512, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=512, bias=True)
    (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_featur

#Combining Vit & ResNet50

In [ ]:
# import numpy as np
# from keras.layers import Dense, BatchNormalization, Dropout
# from keras.preprocessing.image import ImageDataGenerator
# from keras.models import Model, Sequential
# from keras.optimizers import Adam
# from keras.callbacks import EarlyStopping

# from keras.utils import to_categorical
# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# model_resNet50 = ResNet50_Transformer()
# model_ViT = VisionTransformer()

# """LOADING WEIGHTS FROM PREVIOUSLY TRAINED MODELS (see files K_REPORT_MAIN_R50 resp. K_REPORT_MAIN_VGG)"""
# model_resNet50.load_weights("K_R50_T2.h5")
# model_ViT.load_weights("K_VGG_T2.h5")


# """MODEL COMBINATION"""
# # creating the model extracting features from the last layer before the softmax layer.
# vgg_extractor = Model(inputs=model_ViT.input, outputs=model_ViT.get_layer("dense_1024").output)
# r50_extractor = Model(inputs=model_resNet50.input, outputs=model_resNet50.get_layer("dense_1024").output)

# # using the model extractor to generate feature arrays with 1024 features from each model extractor.
# y_train = to_categorical(training_set_V2.classes)
# X_train_m1 = vgg_extractor.predict_generator(training_set_V2, steps = training_set_V2.n)
# X_train_m2 = r50_extractor.predict_generator(training_set_V2, steps = training_set_V2.n)
# X_train = np.concatenate([X_train_m1, X_train_m2], axis=1)

# y_val = to_categorical(validation_set_V2.classes)
# X_val_m1 = vgg_extractor.predict_generator(validation_set_V2, steps = validation_set_V2.n)
# X_val_m2 = r50_extractor.predict_generator(validation_set_V2, steps = validation_set_V2.n)
# X_val = np.concatenate([X_val_m1, X_val_m2], axis=1)

# y_test = to_categorical(test_set_V2.classes)
# X_test_m1 = vgg_extractor.predict_generator(test_set_V2, steps = test_set_V2.n)
# X_test_m2 = r50_extractor.predict_generator(test_set_V2, steps = test_set_V2.n)
# X_test = np.concatenate([X_test_m1, X_test_m2], axis=1)


# """TRAIN & TEST FEATURE EXTRACTION MODEL"""
# # the feature arrays are read into a sequential model directly connecting them to the softmax layer, with
# # some dropout and batch normalization in between.

# # set a seed found to produce a run with consistently high validation accuracy (~.93 +)
# # the specified seed will probably have to be adapted with different train-validation split
# # and loaded model weights of R50 and VGG
# np.random.seed(668)

# opt = Adam(learning_rate=1e-3)
# model = Sequential()
# model.add(Dropout(0.75, input_shape=(2048,)))
# model.add(BatchNormalization())
# model.add(Dense(5, activation="softmax"))
# model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])


# epochs = 200
# model.fit(X_train, y_train, batch_size=32, epochs=epochs, verbose=2, shuffle=True, validation_data=(X_val, y_val))

# """EVALUATE MODEL ON TEST DATA"""
# y_test = np.argmax(y_test, axis=1)
# pred = np.argmax(model.predict(X_test), axis=1)
# print(accuracy_score(y_test, pred))
# print(confusion_matrix(y_test, pred))
# print(classification_report(y_test, pred))

#Training the Model

In [ ]:
import torch.optim as optim

In [ ]:
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

In [ ]:
# # Initialize your Vision Transformer model
# model = ResNet50_Transformer().to(device)

# # Define loss function and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Adjust learning rate as needed

# # Train the model (similar to previous example)
# num_epochs = 10
# for epoch in range(num_epochs):
#     # Training loop
#     model.train()
#     for images, labels in train_loader:
#         images, labels = images.to(device), labels.to(device)
#         optimizer.zero_grad()
#         outputs = model(images)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#     # Calculate training accuracy
#     train_accuracy = correct / total

#     # Validation loop
#     model.eval()
#     with torch.no_grad():
#         val_loss = 0.0
#         correct = 0
#         total = 0
#         for images, labels in val_loader:
#             images, labels = images.to(device), labels.to(device)
#             outputs = model(images)
#             loss = criterion(outputs, labels)
#             val_loss += loss.item()
#             _, predicted = torch.max(outputs, 1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()

#     print(f"Epoch {epoch+1}/{num_epochs}, Training Accuracy: {train_accuracy}, Training Loss: {loss.item()}, Validation Accuracy: {correct/total}")
#     # Save model's state dictionary
#     torch.save(model.state_dict(), '/content/drive/My Drive/resnet50_model_epoch_{}.h5'.format(epoch+1))

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np

# Initialize your Vision Transformer model
model = ResNet50_Transformer().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Adjust learning rate as needed

# Train the model (similar to previous example)
num_epochs = 10
for epoch in range(num_epochs):
    # Training loop
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        correct = 0
        total = 0
        predictions = []
        true_labels = []
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Collect predictions and true labels for confusion matrix
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    # Calculate training accuracy
    train_accuracy = correct / total

    print(f"Epoch {epoch+1}/{num_epochs}, Training Accuracy: {train_accuracy}, Training Loss: {loss.item()}, Validation Accuracy: {correct/total}")
    # Save model's state dictionary
    torch.save(model.state_dict(), '/content/drive/My Drive/resnet50_model_epoch_{}.h5'.format(epoch+1))

    # Calculate confusion matrix
    cm = confusion_matrix(true_labels, predictions)
    print("Confusion Matrix:")
    print(cm)

Epoch 1/10, Training Accuracy: 0.912828947368421, Training Loss: 0.1986999809741974, Validation Accuracy: 0.912828947368421
Confusion Matrix:
[[119   3   0   0   0]
 [ 10 114   2   0   1]
 [  2  13  92   6   0]
 [  4   0   0 107   1]
 [  0   8   3   0 123]]
Epoch 2/10, Training Accuracy: 0.9013157894736842, Training Loss: 0.27588513493537903, Validation Accuracy: 0.9013157894736842
Confusion Matrix:
[[102  19   0   0   1]
 [  1 122   3   0   1]
 [  0  24  86   1   2]
 [  0   2   1 107   2]
 [  0   3   0   0 131]]
Epoch 3/10, Training Accuracy: 0.9407894736842105, Training Loss: 0.022009538486599922, Validation Accuracy: 0.9407894736842105
Confusion Matrix:
[[119   3   0   0   0]
 [ 11 101  10   0   5]
 [  1   2 107   0   3]
 [  0   0   0 112   0]
 [  0   1   0   0 133]]
Epoch 4/10, Training Accuracy: 0.9325657894736842, Training Loss: 0.015569234266877174, Validation Accuracy: 0.9325657894736842
Confusion Matrix:
[[111   8   0   3   0]
 [  3 117   5   1   1]
 [  0   8  99   6   0]
 [  

#Testing the Model

In [ ]:
# Test the model on the test set
model.eval()
with torch.no_grad():
    test_loss = 0.0
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Test Loss: {test_loss}, Test Accuracy: {correct/total}")

#Vit Model

In [ ]:
# Initialize your Vision Transformer model
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vit_model = VisionTransformer(input_channels=3, num_classes=10).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vit_model.parameters(), lr=0.0001)  # Adjust learning rate as needed

# Dummy data loaders for illustration purposes
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [ ]:
# Initialize the correct counter
correct = 0

# Train the model (similar to previous example)
num_epochs = 10
for epoch in range(num_epochs):
    # Initialize total for calculating training accuracy
    total = 0

    # Training loop
    vit_model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = vit_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Calculate training accuracy
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        train_accuracy = correct / total

    # Validation loop
    vit_model.eval()
    with torch.no_grad():
        val_loss = 0.0
        correct = 0
        total = 0
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = vit_model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Epoch {epoch+1}/{num_epochs}, Training Accuracy: {train_accuracy}, Training Loss: {loss.item()}, Validation Accuracy: {correct/total}")
    # Save model's state dictionary
    torch.save(vit_model.state_dict(), '/content/drive/My Drive/viT_model_epoch_{}.h5'.format(epoch+1))

In [ ]:
# Test the model on the test set
vit_model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient calculation as we're only evaluating the model
    test_loss = 0.0  # Initialize the test loss
    correct = 0  # Initialize the number of correct predictions
    total = 0  # Initialize the total number of samples

    # Loop through the test set
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to the device (e.g., GPU)
        outputs = vit_model(images)  # Forward pass

        # Calculate the loss
        loss = criterion(outputs, labels)
        test_loss += loss.item()  # Accumulate the test loss

        # Calculate the number of correct predictions
        _, predicted = torch.max(outputs, 1)  # Get the predicted labels
        total += labels.size(0)  # Increment the total count by the batch size
        correct += (predicted == labels).sum().item()  # Increment the correct count by the number of correct predictions

    # Calculate and print the average test loss and test accuracy
    print(f"Test Loss: {test_loss}, Test Accuracy: {correct/total}")


In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np

model = ResNet50_Transformer().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        correct = 0
        total = 0
        predictions = []
        true_labels = []
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    train_accuracy = correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Training Accuracy: {train_accuracy}, Training Loss: {loss.item()}, Validation Accuracy: {correct/total}")
    torch.save(model.state_dict(), '/content/drive/My Drive/resnet50_model_epoch_{}.h5'.format(epoch+1))
    cm = confusion_matrix(true_labels, predictions)
    print("Confusion Matrix:")
    print(cm)

model.eval()
with torch.no_grad():
    test_loss = 0.0
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f"Test Loss: {test_loss}, Test Accuracy: {correct/total}")

vit_model = VisionTransformer(input_channels=3, num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vit_model.parameters(), lr=0.0001)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
correct = 0

num_epochs = 10
for epoch in range(num_epochs):
    total = 0
    vit_model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = vit_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        train_accuracy = correct / total

    vit_model.eval()
    with torch.no_grad():
        val_loss = 0.0
        correct = 0
        total = 0
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = vit_model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Epoch {epoch+1}/{num_epochs}, Training Accuracy: {train_accuracy}, Training Loss: {loss.item()}, Validation Accuracy: {correct/total}")
    torch.save(vit_model.state_dict(), '/content/drive/My Drive/viT_model_epoch_{}.h5'.format(epoch+1))

vit_model.eval()
with torch.no_grad():
    test_loss = 0.0
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = vit_model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f"Test Loss: {test_loss}, Test Accuracy: {correct/total}")


#Plot Performance

In [ ]:
# def plot_performance(train_losses, val_losses, train_accuracies, val_accuracies):
#     plt.figure(figsize=(12, 5))

#     # Plot losses
#     plt.subplot(1, 2, 1)
#     plt.plot(train_losses, label='Training Loss')
#     plt.plot(val_losses, label='Validation Loss')
#     plt.xlabel('Epoch')
#     plt.ylabel('Loss')
#     plt.title('Training and Validation Loss')
#     plt.legend()

#     # Plot accuracies
#     plt.subplot(1, 2, 2)
#     plt.plot(train_accuracies, label='Training Accuracy')
#     plt.plot(val_accuracies, label='Validation Accuracy')
#     plt.xlabel('Epoch')
#     plt.ylabel('Accuracy')
#     plt.title('Training and Validation Accuracy')
#     plt.legend()

#     plt.tight_layout()
#     plt.show()

In [ ]:
# # Plot performance
# plot_performance(train_losses, val_losses, train_accuracies, val_accuracies)

In [ ]:
targ_vals_int = list()
for i in targ_vals:
  targ_vals_int.append(int(i))

pred_vals_int = list()
for i in pred_vals:
  pred_vals_int.append(int(i))

In [ ]:
pred_vals_nonround_int = list()
for i in pred_vals_nonround:
  pred_vals_nonround_int.append(float(i))

In [ ]:
from sklearn.metrics import confusion_matrix as matrix_print_graph_confusion

import matplotlib.pyplot as plotter
matrix_confuse = matrix_print_graph_confusion(targ_vals_int, pred_vals_int)
print(matrix_confuse)

import seaborn

seaborn.heatmap(matrix_confuse, annot=True)
plotter.xlabel('Values predicted')
plotter.ylabel('Values Expected')
plotter.show()

In [ ]:
tr = [0.1986999809741974,0.27588513493537903, 0.022009538486599922, 0.015569234266877174, 0.4261652231216430]
accuracy = []